---
title: "Limited Dependent Variable Models"
subtitle: "MLE-Mua ha ha ha"
author: Fernando Rios-Avila
jupyter: nbstata
format: 
  revealjs: 
    slide-number: true
    width: 1600
    height: 900
    code-fold: true
    code-overflow: wrap
    echo: true
    css: styles.css 
    chalkboard: true  
---


## What do we mean Limited??


In [ ]:
*| output: false

clear
set obs 2000
gen r1 = runiform()
gen r2 = rchi2(5)/5 
gen r3 = round(rchi2(3))*3
gen r4 = rnormal()
set scheme white2
color_style tableau
histogram r1, name(m1, replace) 
histogram r2, name(m2, replace)
histogram r3, name(m3, replace) width(1)  
histogram r4, name(m4, replace)
graph combine m1 m2 m3 m4
graph export images/fig9_1.png, width(1000) replace

![Limited Dependent variables](images/fig9_1.png)

##

### What do we mean Limited?? {.middle}

- When we think about "limited dependent variable" models, we refer to models when the distribution of the dep.variable is "limited"
  - In other words. The values it can take are restricted! (positive, or only integer), within a range, etc

- Can you still use LRM for them? 
- Will anything change if you do?
- Do we care?

## No we dont, but..

- We dont really care. In fact we have already use LRM on that fashion:
  - LPM: Dep variable was a Dummy
  - Wages: Always positive
  - \# Children: Countable

- But, there are couple of things one should consider.
  1. Models of this kind are usually heteroskedastic by construction. (robust? Weighted?)
  2. Predictions could made no sense. 
  3. There are better models we could use to analyze the data

**Better** under some assumptions

- However, this models cannot be estimated using OLS (there is no "close form solution")
- We ***may*** need to learn a new method: Maximum Likelihood 


## Probits and Logits

- **LPM** are easy, fast, and good for most data analysis (exploration). But they have some limitations.
- Most limitations can be overcome with alternative models: Logit or Probit
- In constrast with LPM (which aims to explain individual outcomes), Logit/probit aims to explain Conditional Probabilities:

$$p(y=1|x) = G(x\beta)$$

- where the function $G()$ makes sure the predicted outcome is always between 0 and 1. 
- Caveat: Because $G()$ is nonlinear, this is a nonlinear model, and marginal effects are harder to estimate.

##

### What to use for $G()$

- Two leading options:

$$logit: G(x\beta) = \frac{\exp{x\beta}}{1+\exp{x\beta}}$$
$$probit: G(x\beta) = \Phi(x\beta)=\int_{-\infty}^{x\beta}\phi(z)dz$$

- But in practice Either will work. Then why the difference?

## 

### Probits and Logits: Latent variables

- It all comes down to the Latent variable!

- Assumption: 
  - Everybody has a latent score on every "binary" decision: The value to a decision $y^*$
    $$y^* = x\beta + e $$

  - If $y^*$ is above certain threshold ($y^*>0$), you "do" something ($y=1$). If not you dont ($y=0$).
- Thus the choice between logit and probit depends on the distribution of $e$.
  - $e$ is normal, then probit
  - $e$ is logistic, then logit


##

### Some Math

Latent Model:

$$ y^* = x\beta + e $$

We aim to measure the probablity of a positive latent. 

$$\begin{aligned}
P(y^*>0|x) & = P(x\beta + e>0|x) \\
& = P( e>- x\beta|x) \\
& = 1 - P( e < - x\beta|x) = 1-G( - x\beta|x) \\
& = G(x\beta)
\end{aligned}
$$

last step valid only if $G()$ is symetrical. 


##

###  Marginal Effects?

- Same as before. The partial derivative!

$$\begin{aligned}
p(y=1|x) &= G(\beta_0 + \beta_1 x_1 +\beta_2 x_2 ) \\
\frac{\partial p(y=1|x)}{\partial x_1} = G'(x\beta)\beta_1=g(x\beta)\beta_1
\end{aligned}
$$

- But if variables are dummies, we need to estimate true effect.

$$\begin{aligned}
p(y=1|x) &= G(\beta_0 + \beta_1 x_1 +\beta_2 D_2 ) \\
\frac{\partial p(y=1|x)}{\partial D_2} = G(\beta_0 + \beta_1 x_1 +\beta_2 )-G(\beta_0 + \beta_1 x_1 )
\end{aligned}
$$

and yes, you could also have interactions, polynomials, etc

## MLE: How does this work?

- MLE: Maximum Likelihood Estimator, is an alternative method to OLS that allows you to estimate parameters in nonlinear models.
- The idea of the method is to "model" the conditional distribution of the data $F(y|x,\theta)$ or $f(y|x,\theta)$, assuming $X's$ are given and modifying values of $\theta$ (distribution parameters).

- $LRM$ **could** be estimated via MLE, but you will need More assumptions:
  - The error $e$ is normal.
- Then "simply" find the parameters for the mean and variance that "maximizes" the probability that data Comes a given distribution.

- In the case of Probit/logit, there is "only" one paramter we need to identify. The conditional probabilty $p(y=1|X)$.
  - Except that we allow this to vary by $X$

## Likelihood function for Logit/probit

$$L_i = G(x\beta)^{y=1}*(1-G(x\beta))^{y=0} 
$$

Under Independence:

$$L_D = L_1 \times L_2 \times \dots L_N
$$

Thus we need to find the $\beta's$ that make $L_D$ the largest.

But because we like sums over products:

$$LL_D = \sum_{i=1}^N log(L_i)
$$

##


In [ ]:
  clear
  set obs 25
  gen r = runiform()<.7
  mata: 
    r = st_data(.,"r")
    ll = J(99,2,0)
    for(i=1;i<=99;i++){
      theta = i/100
      // Log Properties
      ll[i,]= theta,exp(sum(log(theta:^(r:==1) :* (1-theta):^(r:==0))))
    }
  end
  qui getmata ll*=ll , force
  ren ll1 theta
  ren ll2 likelihood
  *scatter likelihood theta 


In [ ]:
*| echo: false
qui:scatter likelihood theta 

## Testing?

- You can test two things:
  - Test coefficients ($\beta$)
  - Test marginal effects ($G'(x\beta)\beta$)
- Both test will most likely agree with each other, but some contradictions may arise.
### How? 
- z-test and/or Wald test: Similar to t-test and Joint F-test we cover before. But, we now make the assumption of normality (not t-distribution)
- Log-Likelihood test. Similar to F-test for restricted and unrestricted model:

  - Estimate both Restricted and unrestricted model. And obtain their Log Likelihoods ($\mathcal{L}_ur$) and ($\mathcal{L}_r$).

  $$LR = 2 (\mathcal{L}_ur-\mathcal{L}_r) \overset{a}\sim \chi^2_q$$

## Stata - Example {.scrollable}


In [ ]:
*| classes: larger
frause mroz, clear
* LPM with Robust Standard errors
qui:reg inlf nwifeinc educ exper expersq age kidslt6 kidsge6, robust
est sto m1
qui:logit inlf nwifeinc educ exper expersq age kidslt6 kidsge6, 
est sto m2a
qui:margins, dydx(*) post
est sto m2b
probit inlf nwifeinc educ exper expersq age kidslt6 kidsge6, 
est sto m3a
qui:margins, dydx(*) post
est sto m3b

In [ ]:
set linesize 255
*| classes: larger
display "Prob Models"
esttab m1 m2a m2b m3a m3b, scalar(r2 ll) cell(b(fmt(%5.3f)) ///
se(par([ ])) p( par(( )) ) )  gap  mtitle(LPM Logit Logit-mfx Probit Probit-mfx)

In [ ]:
*| classes: larger
*| code-fold: false

display "LR test"
qui:probit inlf nwifeinc educ exper expersq age kidslt6 kidsge6 motheduc fatheduc, 
est sto unrestricted
qui:probit inlf nwifeinc educ exper expersq age kidslt6 kidsge6 , 
est sto restricted
lrtest unrestricted restricted

# Break? (5 mins!)

## Censored and Truncated Data

- Logits and Probits, are not the only models that require MLE for estimation. 
  - Among Discrete data models, you also have ologit/oprobit for ordered responses. mlogit/mprobit for unordered ones. Extends on logit/probit.
- There are other interesting cases:
  - When Data is censored.
  - When Data is truncated.

## Three Cases

:::{.panel-tabset}

## Case 1

- $y$ is "conditionally-normal" and is Fully Observed.
- You can estimate the model using OLS or ML


In [ ]:
*| fig-align: center

qui:{
  clear
  set obs 999
  gen p   = _n/(_N+1)
  gen fob = invnormal(p)
}
qui:histogram fob

## Case 2

- Data is observed for everyone, but is "censored" for some. `tobit`
  - Either corner solution (how many hours you study) or Recoded: $y_{obs} = max(c,y^*)$


In [ ]:
*| fig-align: center
*| echo: false*| 
qui: replace fob = -2 if fob<-2
qui:histogram fob, xlabel(-4 (2) 4)

## Case 3

- Below (or above) some threshold, you do not have information on $y$. `truncreg` 
$$y_{obs} = y^* \text{ if } y^*>c$$


In [ ]:
*| echo: false
*| fig-align: center
qui: replace fob = . if fob<=-2
qui:histogram fob, xlabel(-4 (2) 4)

:::

## Estimation: Censored and Corner Solution

If data is censored or corner solution the estimation strategy is based on:

$$\begin{aligned}
L_i &= \frac{1}{\sigma} \phi\left( \frac{y-x\beta}{\sigma} \right) \text{ if } y>c \\
    &= 1-\Phi\left(\frac{x\beta}{\sigma} \right) \text{ if } y\leq c \\
\end{aligned}
$$

If data is truncated, we need to "adjust" the distribution of what is observed

$$\begin{aligned}
L_i &= \frac{1}{\Phi\left( x\beta/\sigma \right)} \frac{1}{\sigma} \phi\left( \frac{y-x\beta}{\sigma} \right) \text{ if } y>c \\  
\end{aligned}
$$

We will put -truncated regression- on the side for now. But see [here](https://stats.oarc.ucla.edu/stata/output/truncated-regression/) for an example.

## Interpretation: It depends!

- What are you interested in analyzing? and what type of data you have?  

:::{.panel-tabset}

## Latent variable

- Easiest Case. Just need to consider the coefficients (as in LRM)

$$
\begin{aligned}
E(y^*|x) &= x\beta \\
\frac{\partial E(y^*|x)}{\partial x } &= \beta_x
\end{aligned}
$$

- The same applies if model was censored.

## $P(y>0|x)$ 

- Its an alternative approach to Probit models, where you are interest in analyzing why is data Not censored, or why is it above some threshold. (why people work)

- Extensive margin effect.
$$
\begin{aligned}
P(y>0|x) &= \Phi\left(\frac{x\beta}{\sigma}\right) \\
\frac{\partial P(y>0|x)}{\partial x } &= \frac{\beta_x}{\sigma} \phi\left(\frac{x\beta}{\sigma}\right)
\end{aligned}
$$

Note: Coefficients $\beta$ need to be Standardized.

## $E(y|y>0,x)$ 

- If corner solution, one may be interested in the effect of those with positive outcomes only. 

- This is the intensive margin effect.
$$
\begin{aligned}
E(y|y>0,x) &= x\beta + \sigma \frac{\phi(x\beta / \sigma )}{\Phi(x\beta / \sigma )} \\
\frac{\partial E(y|y>0,x)}{\partial x } &= \beta_x
\left[ 1-\frac{\phi(x\beta / \sigma )}{\Phi(x\beta / \sigma )} \left( \frac{x\beta }{\sigma }+ \frac{\phi(x\beta / \sigma )}{\Phi(x\beta / \sigma )}\right) \right]
\end{aligned}
$$

## $E(y|x)$ 

- In this case, one may be interested in estimating the expected effect on everyone.
- Combines both Intensive and extensive margin effects. Comparable to OLS.

$$
\begin{aligned}
E(y|x) &= p(y>0|x)*E(y|y>0,x) + (1-p(y>0|x))*0 \\
\frac{\partial E(y|x)}{\partial x } &= \beta_x \Phi(x\beta)
\end{aligned}
$$

:::

## Example {.scrollable}


In [ ]:
frause mroz, clear
qui:tobit hours nwifeinc educ c.exper##c.exper   age kidslt6 kidsge6 , ll(0)
qui:emargins, dydx(*) estore(m1)
qui:emargins, dydx(*) predict(p(0,.)) estore(m2)
qui:emargins, dydx(*) predict(e(0,.)) estore(m3)
qui:emargins, dydx(*) predict(ystar(0,.)) estore(m4)
esttab m1 m2 m3 m4, mtitle(Latent P(y>0) E(y|y>0) E(y) ) b(3) se

## Tobit has problems too

> That simple equation, too much aggregation  
>    Hayek (in [Fear the Boom and Bust](https://www.youtube.com/watch?v=d0nERTFo-Sk))

- Tobit, when addressing corner solutions, aims to explain two different actions (Engagement and intensity) with the same model. However, this may not be appropriate all the time.
  - HW-Examples?
- When this happens, other models may be more appropritate like
  - two part model: (literally model using two equations)
  - Hurdle Model (`craggit` or `churdle`)
- Also...Normality...

## Censored Regression {.scrollable}

- Applies to the same cases as Tobit model. But, it usually refers to Censoring at other points of the distribution (upper censoring? mixed censoring?) 
- Furthermore, applies to cases with different censoring thresholds!
  - Typical Example, Unemployment duration


In [ ]:
qui:frause recid, clear
gen lldur = ldurat             // Lower Limit
gen uudur = ldurat if cens==0  // upper limit = . if censored.
intreg lldur uudur workprg priors tserved felon alcohol drugs black married educ age

## Truncated {.scrollable}

- If Data is simply not there, as shown before, one needs to adjust Estimates.
- marginal effects decisions are similar to Tobit


In [ ]:
*| code-fold: false
frause mroz, clear
qui:truncreg hours nwifeinc educ c.exper##c.exper   age kidslt6 kidsge6 , ll(0)
emargins, dydx(*) estore(m1b)
emargins, dydx(*) predict(e(0,.)) estore(m2b)
esttab m1 m1b m3 m2b, mtitle(Lat-Tobit Lat-Trunc E(y>0)-Tobit E(y>0)-Trunc ) b(3) se

# Break?

## Poisson

- Some times, Data may be non-negative, and/or countable. OLS works well, but we could do better

- With Count data, some data transformations (logs) are not possible, because of the zeroes. 

- So instead of assuming $y|x \sim N(\mu_x,\sigma)$, one could assume $y|x \sim poisson(\mu_x)$

$$P(y=k,\mu_x) = \frac{\mu_x^k e ^{-\mu_x}}{k!} \text{ with } \mu_x=\exp(x\beta)$$

- For a Poisson:
  - $E(y|x) = \exp{x\beta}$ and $Var(y|x) = \exp{x\beta}$

- As hinted before, Count data is heteroskedastic. And Poisson assumes some structure to that.

## 

- Also convinient that Poisson models are very easy to interpret! (just like Log-lin models)  
  After estimation:

$$\frac{\Delta \% E(y|x)}{\Delta x} \simeq \beta_x \times 100 \text{ or } (\exp \beta_x-1)\times 100 $$

- Other points.
  - The variance imposed in Poisson is very restrictive. This is a problem for Variance estimation!  
    Solution: use Robust Standard Errors!
  - Like LRM, poisson is robust to errors when modeling the conditional mean.
  - Poisson is a very good alternative for continuous data too (if using Robust SE)
    - Wage models, trade models

## Example{.scrollable}


In [ ]:
*| code-fold: false
frause crime1, clear
qui: reg narr86 pcnv avgsen tottime ptime86 qemp86 inc86 black hispan born60
est sto m1
qui:poisson narr86 pcnv avgsen tottime ptime86 qemp86 inc86 black hispan born60, robust
est sto m2
qui:emargins, dydx(*) estore(m3)
esttab m1 m2 m3, se b(3) mtitle(LRM Poisson Poisson-mfx) ///
keep(pcnv ptime86  qemp86 inc86 black hispan) label varwidth(20) wrap

## Other Methods of interest

- MLE opens the door to other methods that may be more approriate to analyze data
- They may even be able to handle otherwise unsolvable data problems. 
  - ologit, oprobit: Ordered qualitative variables
  - mlogit, mprobit: Unordered Qualitative variables
  - heckman: Endogenous Sample Selection
  - fractional regression model: When the depvariable is an index
  - etc etc
- Worth knowing, but not for the exam!

# Thats all. 
Next class: Pool and Basic Panel data analysis